In [1]:

import warnings
import pandas as pd
from utils.Copernicus import AdvancedCopernicus
import datetime
import os

# Ignore SettingWithCopyWarning:
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)



# Display all columns
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

/home/marco/venv/.main/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
filenames = os.listdir("data")
print(filenames)

filename = "b'KielHoltenauTG'_hourly.json"

df = pd.read_json(f"data/{filename}", lines=True)
df['TIME'] = pd.to_datetime(df['TIME'])
df

,TIME,DEPTH,TIME_QC,DEPH,LATITUDE,LONGITUDE,SLEV,SLEV_QC
0,2023-01-01 01:00:00,0.0,1.0,0.0,54.382999,10.204,0.019667,1.0
1,2023-01-01 02:00:00,0.0,1.0,0.0,54.382999,10.204,-0.035333,1.0
2,2023-01-01 03:00:00,0.0,1.0,0.0,54.382999,10.204,-0.118667,1.0
3,2023-01-01 04:00:00,0.0,1.0,0.0,54.382999,10.204,-0.163667,1.0
4,2023-01-01 05:00:00,0.0,1.0,0.0,54.382999,10.204,-0.100333,1.0
...,...,...,...,...,...,...,...,...
18954,2025-02-28 19:00:00,0.0,1.0,0.0,54.382999,10.204,0.041333,1.0
18955,2025-02-28 20:00:00,0.0,1.0,0.0,54.382999,10.204,0.008000,1.0
18956,2025-02-28 21:00:00,0.0,1.0,0.0,54.382999,10.204,-0.013667,1.0
18957,2025-02-28 22:00:00,0.0,1.0,0.0,54.382999,10.204,-0.027000,1.0


In [7]:
df_anomalies = df.copy()
df_anomalies['anomaly'] = df_anomalies['SLEV'].apply(lambda x: 1 if x > 1.0 else 0) 
df_anomalies

,TIME,DEPTH,TIME_QC,DEPH,LATITUDE,LONGITUDE,SLEV,SLEV_QC,anomaly
0,2023-01-01 01:00:00,0.0,1.0,0.0,54.382999,10.204,0.019667,1.0,0
1,2023-01-01 02:00:00,0.0,1.0,0.0,54.382999,10.204,-0.035333,1.0,0
2,2023-01-01 03:00:00,0.0,1.0,0.0,54.382999,10.204,-0.118667,1.0,0
3,2023-01-01 04:00:00,0.0,1.0,0.0,54.382999,10.204,-0.163667,1.0,0
4,2023-01-01 05:00:00,0.0,1.0,0.0,54.382999,10.204,-0.100333,1.0,0
...,...,...,...,...,...,...,...,...,...
18954,2025-02-28 19:00:00,0.0,1.0,0.0,54.382999,10.204,0.041333,1.0,0
18955,2025-02-28 20:00:00,0.0,1.0,0.0,54.382999,10.204,0.008000,1.0,0
18956,2025-02-28 21:00:00,0.0,1.0,0.0,54.382999,10.204,-0.013667,1.0,0
18957,2025-02-28 22:00:00,0.0,1.0,0.0,54.382999,10.204,-0.027000,1.0,0


In [10]:

# Identifikation von Anomalien mit Z-Score > 3
df_anomalies_grouped = df_anomalies.loc[df_anomalies['anomaly'] == 1, ['TIME', 'SLEV', 'anomaly']].groupby(by='TIME').mean().reset_index()
df_anomalies_grouped

,TIME,SLEV,anomaly
0,2023-02-25 11:00:00,1.006333,1.0
1,2023-02-25 12:00:00,1.073000,1.0
2,2023-02-25 13:00:00,1.101333,1.0
3,2023-02-25 14:00:00,1.148000,1.0
4,2023-02-25 15:00:00,1.146333,1.0
...,...,...,...
88,2024-02-11 13:00:00,1.046333,1.0
89,2024-02-11 14:00:00,1.049667,1.0
90,2024-02-11 15:00:00,1.053000,1.0
91,2024-02-11 16:00:00,1.033000,1.0


In [21]:
import plotly.graph_objects as go
import pandas as pd
import datetime



sla_mean = df['SLEV'].mean()

# Erstelle die Plotly-Figur
fig = go.Figure()


# Zukunft als gestrichelte Linie (Forecast)
fig.add_trace(go.Scatter(
    x=df['TIME'], y=df['SLEV'], 
    mode='lines', name='Sea Level Forecast', 
    line=dict(color='blue')
))

# Plot Mean 
fig.add_trace(go.Scatter(
    x=[df['TIME'].min(), df['TIME'].max()], y=[sla_mean, sla_mean],
    mode='lines', name='Mean',
    line=dict(color='red', dash='dash')
))



# Anomalien als rote Punkte
fig.add_trace(go.Scatter(
    x=df_anomalies_grouped['TIME'], y=df_anomalies_grouped['SLEV'], 
    mode='markers', name='Detected Storm Surge', 
    marker=dict(color='red', size=8)
))

# FARBBEREICHE für verschiedene Wasserstandshöhen
fig.add_shape(type="rect", x0=df['TIME'].min(), x1=df['TIME'].max(),
              y0=1, y1=1.25, fillcolor="yellow",
              opacity=0.3, layer="below", line_width=0, name='Low Anomaly')

fig.add_shape(type="rect", x0=df['TIME'].min(), x1=df['TIME'].max(),
              y0=1.25, y1=1.5, fillcolor="orange",
              opacity=0.3, layer="below", line_width=0)

fig.add_shape(type="rect", x0=df['TIME'].min(), x1=df['TIME'].max(),
              y0=1.5, y1=2, fillcolor="red",
              opacity=0.3, layer="below", line_width=0)

fig.add_shape(type="rect", x0=df['TIME'].min(), x1=df['TIME'].max(),
              y0=2, y1=2.5, fillcolor="darkred",
              opacity=0.3, layer="below", line_width=0)


fig.add_annotation(x=df['TIME'].median(), y=1.125,
                   text="Strumflut", showarrow=False,
                   font=dict(size=12, color="black"),
                   bgcolor="yellow", opacity=0.7)

fig.add_annotation(x=df['TIME'].median(), y=1.375,
                   text="mittlere Sturmflut", showarrow=False,
                   font=dict(size=12, color="black"),
                   bgcolor="orange", opacity=0.7)

fig.add_annotation(x=df['TIME'].median(), y=1.75,
                   text="schwere Sturmflut", showarrow=False,
                   font=dict(size=12, color="white"),
                   bgcolor="red", opacity=0.7)

fig.add_annotation(x=df['TIME'].median(), y=2.25,
                   text="sehr schwere Sturmflut", showarrow=False,
                   font=dict(size=12, color="white"),
                   bgcolor="darkred", opacity=0.7)


# Layout anpassen
fig.update_layout(
    title=f'Sea Level in Kiel Holtenau from {df["TIME"].min()} to {df["TIME"].max()}',
    xaxis_title='Time',
    yaxis_title='Waterlevel [m]',
    width=1500,
    height=800,
    xaxis=dict(
        tickangle=45, 
        tickformat='%Y-%m-%d %H:%M:%S',
        showline=True,
        
        rangeslider=dict(visible=False),
        type='date'
    ),
    legend=dict(
        x=1.02,  # Position rechts außerhalb des Plots
        y=1,  
        xanchor='left',  # Legende am linken Rand der Position ausrichten
        yanchor='top',
        bgcolor='rgba(255,255,255,0.6)',  # Hintergrund für bessere Lesbarkeit
        bordercolor='black',
        borderwidth=1
    ),
    template='plotly_white'
)

fig.show()
